This notebook shows how to use [dvc](https://dvc.org/) [experiments](https://github.com/iterative/dvc/wiki/Experiments) in model development. This example uses the [MNIST](http://yann.lecun.com/exdb/mnist/) data of handwritten digits and builds a classification model to predict the digit (0-9) in each image. The model is built in [pytorch](https://pytorch.org/) as a convolutional neural network with a simplified architecture, which should be able to quickly run on most computers.

### Get started

To get started, clone this repository and navigate to it.

The only other prerequisite is [conda](https://docs.conda.io/projects/conda/en/latest/user-guide/install/). Once conda is installed, create a virtual environment from the existing `environment.yaml` file and activate it:

```bash
conda env create -f environment.yml
conda activate dvc
```

If you want to run this notebook directly, do so after activating the conda environment.

Finally, initialize dvc and enable the experiments feature:

In [43]:
%%bash
dvc init -f
dvc config --global core.experiments true

Initialized DVC repository.

You can now commit the changes to git.

+---------------------------------------------------------------------+
|                                                                     |
|        DVC has enabled anonymous aggregate usage analytics.         |
|     Read the analytics documentation (and how to opt-out) here:     |
|             <https://dvc.org/doc/user-guide/analytics>              |
|                                                                     |
+---------------------------------------------------------------------+

What's next?
------------
- Check out the documentation: <https://dvc.org/doc>
- Get help and share ideas: <https://dvc.org/chat>
- Star us on GitHub: <https://github.com/iterative/dvc>


### Establish the pipeline DAG

Before experimenting, a dvc pipeline must be established (see the docs if you are new to dvc). Review the contents of `dvc.yaml` below to see the pipeline.

In [44]:
%%bash
cat dvc.yaml

stages:
  download:
    cmd: python download.py
    deps:
    - download.py
    outs:
    - data/MNIST
  train:
    cmd: python train.py
    deps:
    - data/MNIST
    - train.py
    params:
    - lr
    - weight_decay
    outs:
    - model.pt:
        checkpoint: true
    metrics:
    - metrics.yaml


The download stage gets the data using the `download.py` script. The train stage performs model training and evaluation on the downloaded data using the `train.py` script. The train stage uses the lr and weight_decay metrics defined in `params.yaml`. The model output is saved to `model.pt`, and the metrics are saved to `metrics.yaml`.

Execute the download stage to get the data.

In [45]:
%%bash
dvc repro download

Running stage 'download':
> python download.py
Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw
Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw
Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw
Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw
Processing...
Done!
Migrating lock file 'dvc.lock' from v1 to v2
Updating lock file 'dvc.lock'

To track the changes with git, run:

	git add dvc.lock
Use `dvc push` to send your updates to remote storage.


 97%|█████████▋| 9609216/9912422 [00:03<00:00, 4430304.19it/s]
0it [00:00, ?it/s]
  0%|          | 0/28881 [00:00<?, ?it/s]

0it [00:00, ?it/s]

  1%|          | 16384/1648877 [00:00<00:12, 125983.52it/s]

 16%|█▌        | 262144/1648877 [00:00<00:01, 1263258.51it/s]

 35%|███▍      | 573440/1648877 [00:00<00:00, 2023186.01it/s]

 48%|████▊     | 794624/1648877 [00:00<00:00, 2039029.91it/s]

 68%|██████▊   | 1114112/1648877 [00:00<00:00, 2395510.57it/s]

 85%|████████▍ | 1400832/1648877 [00:00<00:00, 2547978.78it/s]


0it [00:00, ?it/s]/home/dave/anaconda3/envs/dvc-exp-mnist/lib/python3.8/site-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for 

**IMPORTANT:** Be sure to run the `git add` command above and also `git commit` before running experiments. Anytime you modify the pipeline, be sure to `dvc repro` and track changes with git before running experiments.

In [46]:
%%bash
git add dvc.lock data/.gitignore
git commit -m "download data"

[main 77a93c2] download data
 2 files changed, 34 insertions(+), 12 deletions(-)
 rewrite dvc.lock (100%)


### Run an experiment

Run an experiment with the default parameters defined in `params.yaml`.

In [47]:
%%bash
dvc exp run

Stage 'download' didn't change, skipping
Running stage 'train':
> python train.py
Updating lock file 'dvc.lock'
Checkpoint experiment iteration 'a4cce79'.
Updating lock file 'dvc.lock'
Checkpoint experiment iteration 'e622bc1'.

To track the changes with git, run:

	git add data/MNIST .gitignore dvc.lock params.yaml train.py dvc.yaml

Reproduced experiment(s): exp-17fa7
Experiment results have been applied to your workspace.

To promote an experiment to a Git branch run:

	dvc exp branch <exp>



Review the output of the run, including identifying hashes, metrics, and parameters:

In [49]:
%%bash
dvc exp show

┏━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━━━┓
┃ Experiment    ┃ Created  ┃    acc ┃   loss ┃ lr    ┃ weight_decay ┃
┡━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━━━┩
│ workspace     │ -        │  0.144 │ 2.2961 │ 0.001 │ 0.1          │
│ main          │ 11:12 AM │      - │      - │ 0.001 │ 0            │
│ │ ╓ exp-17fa7 │ 11:12 AM │  0.144 │ 2.2961 │ 0.001 │ 0.1          │
│ ├─╨ a4cce79   │ 11:12 AM │ 0.1695 │ 2.2989 │ 0.001 │ 0.1          │
└───────────────┴──────────┴────────┴────────┴───────┴──────────────┘


Note that two experiments were run. These are checkpoints for the run. It's not necessary to have checkpoints for experiments, but they can be helpful for models that may be run for a number of epochs. See below for more information about how checkpoints work.

### Experiment with different parameters

Experiments can be run and compared with different parameters.

In [50]:
%%bash
dvc exp run --params weight_decay=0.1

Stage 'download' didn't change, skipping
Running stage 'train':
> python train.py
Updating lock file 'dvc.lock'
Checkpoint experiment iteration '0569f30'.
Updating lock file 'dvc.lock'
Checkpoint experiment iteration 'e28ad6f'.

To track the changes with git, run:

	git add params.yaml dvc.lock data/MNIST train.py dvc.yaml

Reproduced experiment(s): exp-1ab12
Experiment results have been applied to your workspace.

To promote an experiment to a Git branch run:

	dvc exp branch <exp>



In [8]:
%%bash
dvc exp show

┏━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━━━┓
┃ Experiment    ┃ Created  ┃    acc ┃   loss ┃ lr    ┃ weight_decay ┃
┡━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━━━┩
│ workspace     │ -        │      - │      - │ 0.001 │ 0            │
│ main          │ 10:06 AM │      - │      - │ 0.001 │ 0            │
│ │ ╓ exp-fccd3 │ 10:09 AM │ 0.1025 │ 2.3051 │ 0.001 │ 0.1          │
│ ├─╨ 31e8e72   │ 10:08 AM │ 0.0947 │  2.308 │ 0.001 │ 0.1          │
│ │ ╓ exp-d0a96 │ 10:07 AM │ 0.2445 │ 2.2616 │ 0.001 │ 0            │
│ ├─╨ d025f19   │ 10:07 AM │ 0.1895 │ 2.2934 │ 0.001 │ 0            │
└───────────────┴──────────┴────────┴────────┴───────┴──────────────┘


Increasing `weight_decay` helped a bit. Next, try different `lr` parameters.

Experiments can be added in bulk to the queue and executed on demand (see the `-j` flag for parallel execution!).

In [9]:
%%bash
dvc exp run --params lr=0.01 --queue
dvc exp run --params lr=0.1 --queue

Queued experiment '0c2ae1f' for future execution.
Queued experiment '927d26e' for future execution.


In [10]:
%%bash
dvc exp show

┏━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━━━┓
┃ Experiment    ┃ Created  ┃    acc ┃   loss ┃ lr    ┃ weight_decay ┃
┡━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━━━┩
│ workspace     │ -        │      - │      - │ 0.001 │ 0            │
│ main          │ 10:06 AM │      - │      - │ 0.001 │ 0            │
│ │ ╓ exp-fccd3 │ 10:09 AM │ 0.1025 │ 2.3051 │ 0.001 │ 0.1          │
│ ├─╨ 31e8e72   │ 10:08 AM │ 0.0947 │  2.308 │ 0.001 │ 0.1          │
│ │ ╓ exp-d0a96 │ 10:07 AM │ 0.2445 │ 2.2616 │ 0.001 │ 0            │
│ ├─╨ d025f19   │ 10:07 AM │ 0.1895 │ 2.2934 │ 0.001 │ 0            │
│ ├── *927d26e  │ 10:10 AM │      - │      - │ 0.1   │ 0            │
│ └── *0c2ae1f  │ 10:10 AM │      - │      - │ 0.01  │ 0            │
└───────────────┴──────────┴────────┴────────┴───────┴──────────────┘


In [11]:
%%bash
dvc exp run --run-all

Stage 'download' didn't change, skipping
Running stage 'train' with command:
	python train.py
Updating lock file 'dvc.lock'
Checkpoint experiment iteration 'd4db485'.
Updating lock file 'dvc.lock'
Checkpoint experiment iteration '046300d'.

Reproduced experiment(s): exp-7edff
To promote an experiment to a Git branch run:

	dvc exp branch <exp>

To apply the results of an experiment to your workspace run:

	dvc exp apply <exp>


In [12]:
%%bash
dvc exp show

┏━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━━━┓
┃ Experiment    ┃ Created  ┃    acc ┃   loss ┃ lr    ┃ weight_decay ┃
┡━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━━━┩
│ workspace     │ -        │      - │      - │ 0.001 │ 0            │
│ main          │ 10:06 AM │      - │      - │ 0.001 │ 0            │
│ │ ╓ exp-7edff │ 10:10 AM │ 0.5774 │ 1.6223 │ 0.01  │ 0            │
│ ├─╨ d4db485   │ 10:10 AM │ 0.1157 │ 2.0914 │ 0.01  │ 0            │
│ │ ╓ exp-fccd3 │ 10:09 AM │ 0.1025 │ 2.3051 │ 0.001 │ 0.1          │
│ ├─╨ 31e8e72   │ 10:08 AM │ 0.0947 │  2.308 │ 0.001 │ 0.1          │
│ │ ╓ exp-d0a96 │ 10:07 AM │ 0.2445 │ 2.2616 │ 0.001 │ 0            │
│ ├─╨ d025f19   │ 10:07 AM │ 0.1895 │ 2.2934 │ 0.001 │ 0            │
│ ├── *927d26e  │ 10:10 AM │      - │      - │ 0.1   │ 0            │
│ └── *0c2ae1f  │ 10:10 AM │      - │      - │ 0.01  │ 0            │
└───────────────┴──────────┴────────┴────────┴───────┴──────────────┘


**FIXME:** Running queued experiments runs new experiments, but not the same ones that were in the queue for `dvc exp show` (nor with the same parameters specified there). The queued experiments remain in the queue.

In [14]:
%%bash
cat params.yaml

lr: 0.001
weight_decay: 0


In [16]:
%%bash
dvc exp run --run-all

Stage 'download' didn't change, skipping
Running stage 'train' with command:
	python train.py
Updating lock file 'dvc.lock'
Checkpoint experiment iteration '0922cba'.
Updating lock file 'dvc.lock'
Checkpoint experiment iteration '36442f5'.

Reproduced experiment(s): exp-688c1
To promote an experiment to a Git branch run:

	dvc exp branch <exp>

To apply the results of an experiment to your workspace run:

	dvc exp apply <exp>


In [17]:
%%bash
dvc exp show

┏━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━━━┓
┃ Experiment    ┃ Created  ┃    acc ┃   loss ┃ lr    ┃ weight_decay ┃
┡━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━━━┩
│ workspace     │ -        │      - │      - │ 0.001 │ 0            │
│ main          │ 10:06 AM │      - │      - │ 0.001 │ 0            │
│ │ ╓ exp-688c1 │ 10:24 AM │ 0.3938 │ 1.6869 │ 0.01  │ 0            │
│ ├─╨ 0922cba   │ 10:23 AM │ 0.2464 │ 2.0991 │ 0.01  │ 0            │
│ │ ╓ exp-7edff │ 10:10 AM │ 0.5774 │ 1.6223 │ 0.01  │ 0            │
│ ├─╨ d4db485   │ 10:10 AM │ 0.1157 │ 2.0914 │ 0.01  │ 0            │
│ │ ╓ exp-fccd3 │ 10:09 AM │ 0.1025 │ 2.3051 │ 0.001 │ 0.1          │
│ ├─╨ 31e8e72   │ 10:08 AM │ 0.0947 │  2.308 │ 0.001 │ 0.1          │
│ │ ╓ exp-d0a96 │ 10:07 AM │ 0.2445 │ 2.2616 │ 0.001 │ 0            │
│ ├─╨ d025f19   │ 10:07 AM │ 0.1895 │ 2.2934 │ 0.001 │ 0            │
│ ├── *927d26e  │ 10:10 AM │      - │      - │ 0.1   │ 0            │
│ └── *0c2ae1f  │ 10

Running `dvc exp run --run-all` again repeats the same process of generating new experiments but not the ones specified in the queue.

Next, let's test that parallel execution works:

In [20]:
%%bash
dvc exp run --run-all -j 2

Stage 'download' didn't change, skipping
Running stage 'train' with command:
	python train.py
Updating lock file 'dvc.lock'
Checkpoint experiment iteration '381b51d'.
Updating lock file 'dvc.lock'
Checkpoint experiment iteration '2ea74c6'.

Reproduced experiment(s): exp-b8d47
To promote an experiment to a Git branch run:

	dvc exp branch <exp>

To apply the results of an experiment to your workspace run:

	dvc exp apply <exp>


### Iteratively train using checkpoints

Use checkpoints to periodically save the model during training (as shown above), and to resume training from previously saved state. Resume training the experiment with the best accuracy.

In [22]:
%%bash
dvc exp show --sort-by acc --include-metrics acc

┏━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━━━┓
┃ Experiment    ┃ Created  ┃    acc ┃ lr    ┃ weight_decay ┃
┡━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━━━┩
│ workspace     │ -        │      - │ 0.001 │ 0            │
│ main          │ 10:06 AM │      - │ 0.001 │ 0            │
│ │ ╓ exp-fccd3 │ 10:09 AM │ 0.1025 │ 0.001 │ 0.1          │
│ ├─╨ 31e8e72   │ 10:08 AM │ 0.0947 │ 0.001 │ 0.1          │
│ │ ╓ exp-d0a96 │ 10:07 AM │ 0.2445 │ 0.001 │ 0            │
│ ├─╨ d025f19   │ 10:07 AM │ 0.1895 │ 0.001 │ 0            │
│ │ ╓ exp-688c1 │ 10:24 AM │ 0.3938 │ 0.01  │ 0            │
│ ├─╨ 0922cba   │ 10:23 AM │ 0.2464 │ 0.01  │ 0            │
│ │ ╓ exp-b8d47 │ 10:27 AM │ 0.4016 │ 0.01  │ 0            │
│ ├─╨ 381b51d   │ 10:27 AM │ 0.2116 │ 0.01  │ 0            │
│ │ ╓ exp-7edff │ 10:10 AM │ 0.5774 │ 0.01  │ 0            │
│ ├─╨ d4db485   │ 10:10 AM │ 0.1157 │ 0.01  │ 0            │
│ ├── *927d26e  │ 10:10 AM │      - │ 0.1   │ 0            │
│ └── *0c2ae1f  │ 10:10 

Apply the top experiment.

In [26]:
%%bash
dvc exp apply exp-fccd3

ERROR: 'exp-fccd3' does not appear to be an experiment commit.: Experiment derived from 'invalid commit', expected '8e8a8f2'.


CalledProcessError: Command 'b'dvc exp apply exp-fccd3\n'' returned non-zero exit status 255.

**FIXME**: That didn't work. What are these lines with `exp-` at the start?

Let's try the other commit associated with these parameters.

In [27]:
%%bash
dvc exp apply 31e8e72

Changes for experiment '31e8e72' have been applied to your current workspace.


Next, resume iterations from this experiment.

In [28]:
%%bash
dvc exp res

Stage 'download' didn't change, skipping
Running stage 'train' with command:
	python train.py
Updating lock file 'dvc.lock'
Checkpoint experiment iteration '5c4438e'.
Updating lock file 'dvc.lock'
Checkpoint experiment iteration 'a7735a9'.

Reproduced experiment(s): exp-b8d47
To promote an experiment to a Git branch run:

	dvc exp branch <exp>

To apply the results of an experiment to your workspace run:

	dvc exp apply <exp>


In [29]:
%%bash
dvc exp show

┏━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━━━┓
┃ Experiment    ┃ Created  ┃    acc ┃   loss ┃ lr    ┃ weight_decay ┃
┡━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━━━┩
│ workspace     │ -        │ 0.1025 │ 2.3051 │ 0.001 │ 0.1          │
│ main          │ 10:06 AM │      - │      - │ 0.001 │ 0            │
│ │ ╓ exp-b8d47 │ 10:36 AM │ 0.6693 │ 1.1839 │ 0.01  │ 0            │
│ │ ╟ 5c4438e   │ 10:35 AM │ 0.5445 │ 1.4061 │ 0.01  │ 0            │
│ │ ╟ 2ea74c6   │ 10:27 AM │ 0.4016 │ 1.7229 │ 0.01  │ 0            │
│ ├─╨ 381b51d   │ 10:27 AM │ 0.2116 │ 2.0748 │ 0.01  │ 0            │
│ │ ╓ exp-688c1 │ 10:24 AM │ 0.3938 │ 1.6869 │ 0.01  │ 0            │
│ ├─╨ 0922cba   │ 10:23 AM │ 0.2464 │ 2.0991 │ 0.01  │ 0            │
│ │ ╓ exp-7edff │ 10:10 AM │ 0.5774 │ 1.6223 │ 0.01  │ 0            │
│ ├─╨ d4db485   │ 10:10 AM │ 0.1157 │ 2.0914 │ 0.01  │ 0            │
│ │ ╓ exp-fccd3 │ 10:09 AM │ 0.1025 │ 2.3051 │ 0.001 │ 0.1          │
│ ├─╨ 31e8e72   │ 10

**FIXME**: `dvc exp res` ran additional experiments, but they aren't using the same parameters to the `31e8e72` parameters that were applied.

In [30]:
%%bash
cat params.yaml

lr: 0.001
weight_decay: 0.1


`dvc exp apply 31e8e72` checkout out that experiment's parameters into the workspace, so it's unclear why `dvc exp res` doesn't use those parameters.

In [39]:
%%bash
dvc exp diff 5c4438e 31e8e72

Path          Metric    Value    Change
metrics.yaml  acc       0.0947   -0.4498
metrics.yaml  loss      2.308    0.90187

Path         Param         Value    Change
params.yaml  lr            0.001    -0.009
params.yaml  weight_decay  0.1      0.1



`dvc exp diff` shows that the additional checkpoints had worse performance (probably because they weren't using the expected parameters).

### Persist models

Additonal epochs didn't improve accuracy, so commit the model iteration with peak accuracy. Checkout the experiment rev in dvc and then commit to git.

In [34]:
%%bash
dvc exp branch 31e8e72 1.11.10

Git branch '1.11.10' has been created from experiment 'exp-fccd3'.
To switch to the new branch run:

	git checkout 1.11.10


In [35]:
%%bash
dvc exp show

┏━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━━━┓
┃ Experiment    ┃ Created  ┃    acc ┃   loss ┃ lr    ┃ weight_decay ┃
┡━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━━━┩
│ workspace     │ -        │ 0.1025 │ 2.3051 │ 0.001 │ 0.1          │
│ main          │ 10:06 AM │      - │      - │ 0.001 │ 0            │
│ │ ╓ exp-b8d47 │ 10:36 AM │ 0.6693 │ 1.1839 │ 0.01  │ 0            │
│ │ ╟ 5c4438e   │ 10:35 AM │ 0.5445 │ 1.4061 │ 0.01  │ 0            │
│ │ ╟ 2ea74c6   │ 10:27 AM │ 0.4016 │ 1.7229 │ 0.01  │ 0            │
│ ├─╨ 381b51d   │ 10:27 AM │ 0.2116 │ 2.0748 │ 0.01  │ 0            │
│ │ ╓ exp-688c1 │ 10:24 AM │ 0.3938 │ 1.6869 │ 0.01  │ 0            │
│ ├─╨ 0922cba   │ 10:23 AM │ 0.2464 │ 2.0991 │ 0.01  │ 0            │
│ │ ╓ exp-7edff │ 10:10 AM │ 0.5774 │ 1.6223 │ 0.01  │ 0            │
│ ├─╨ d4db485   │ 10:10 AM │ 0.1157 │ 2.0914 │ 0.01  │ 0            │
│ │ ╓ 1.11.10   │ 10:09 AM │ 0.1025 │ 2.3051 │ 0.001 │ 0.1          │
│ ├─╨ 31e8e72   │ 10

### Clean up experiments

In [40]:
%%bash
dvc exp gc --workspace

CalledProcessError: Command 'b'dvc exp gc --workspace\n'' returned non-zero exit status 1.

**FIXME**: `dvc exp gc --workspace` did not work.

In [41]:
%%bash
dvc exp show

┏━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━━━┓
┃ Experiment    ┃ Created  ┃    acc ┃   loss ┃ lr    ┃ weight_decay ┃
┡━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━━━┩
│ workspace     │ -        │ 0.1025 │ 2.3051 │ 0.001 │ 0.1          │
│ main          │ 10:06 AM │      - │      - │ 0.001 │ 0            │
│ │ ╓ exp-b8d47 │ 10:36 AM │ 0.6693 │ 1.1839 │ 0.01  │ 0            │
│ │ ╟ 5c4438e   │ 10:35 AM │ 0.5445 │ 1.4061 │ 0.01  │ 0            │
│ │ ╟ 2ea74c6   │ 10:27 AM │ 0.4016 │ 1.7229 │ 0.01  │ 0            │
│ ├─╨ 381b51d   │ 10:27 AM │ 0.2116 │ 2.0748 │ 0.01  │ 0            │
│ │ ╓ exp-688c1 │ 10:24 AM │ 0.3938 │ 1.6869 │ 0.01  │ 0            │
│ ├─╨ 0922cba   │ 10:23 AM │ 0.2464 │ 2.0991 │ 0.01  │ 0            │
│ │ ╓ exp-7edff │ 10:10 AM │ 0.5774 │ 1.6223 │ 0.01  │ 0            │
│ ├─╨ d4db485   │ 10:10 AM │ 0.1157 │ 2.0914 │ 0.01  │ 0            │
│ │ ╓ 1.11.10   │ 10:09 AM │ 0.1025 │ 2.3051 │ 0.001 │ 0.1          │
│ ├─╨ 31e8e72   │ 10